<a href="https://colab.research.google.com/github/Mahika474/Mahika474/blob/main/Autonomous_Delivery_Agent_CLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import argparse
import time
from environment import GridCity
from agent import DeliveryAgent

def main():
    """
    Main function to run the autonomous delivery agent simulation.
    Parses command-line arguments to select the map, algorithm, and agent goals.
    """
    parser = argparse.ArgumentParser(description="Run Autonomous Delivery Agent Simulation")
    parser.add_argument("map", help="Path to the map file (.grid)")
    parser.add_argument("algorithm", choices=['ucs', 'astar', 'replan'], help="Algorithm to use (ucs, astar, replan)")

    args = parser.parse_args()

    # --- Load Environment ---
    print(f"Loading map from {args.map}...")
    environment = GridCity(args.map)
    start_node = environment.start
    goal_node = environment.goal
    print(f"Start: {start_node}, Goal: {goal_node}")
    print("Map loaded successfully.")
    # environment.display() # Optional: display the map

    # --- Initialize Agent ---
    agent = DeliveryAgent(environment)

    # --- Run Selected Algorithm ---
    path = None
    expanded_nodes = 0

    start_time = time.time()

    if args.algorithm == 'ucs':
        print("\nRunning Uniform-Cost Search...")
        path, expanded_nodes = agent.uniform_cost_search(start_node, goal_node)

    elif args.algorithm == 'astar':
        print("\nRunning A* Search...")
        path, expanded_nodes = agent.a_star_search(start_node, goal_node)

    elif args.algorithm == 'replan':
        print("\nRunning Simulation with Replanning...")
        # This is a proof-of-concept simulation loop for dynamic replanning
        current_pos = start_node
        path = [current_pos]
        total_expanded_nodes = 0

        # Initial plan using A*
        print(f"[Time 0] Planning initial path from {current_pos} to {goal_node}")
        current_plan, nodes_exp = agent.a_star_search(current_pos, goal_node)
        total_expanded_nodes += nodes_exp

        if not current_plan:
            print("No initial path found!")
            return

        print(f"Initial plan: {current_plan}")

        # Simulation loop
        simulation_time = 0
        while current_pos != goal_node:
            simulation_time += 1

            # Move agent one step along the plan
            if not current_plan:
                print(f"[Time {simulation_time}] Stuck! No path found.")
                break

            next_step = current_plan.pop(0)

            # Check if the next step is now blocked by a dynamic obstacle
            if environment.is_obstacle(next_step, simulation_time):
                print(f"--- DYNAMIC OBSTACLE DETECTED! ---")
                print(f"[Time {simulation_time}] Obstacle at {next_step}. Current position: {current_pos}.")
                print("Replanning path...")

                # Here you would implement your local search (hill-climbing, etc.)
                # For this proof-of-concept, we just re-run A* from the current position.
                new_plan, nodes_exp = agent.a_star_search(current_pos, goal_node, current_time=simulation_time)
                total_expanded_nodes += nodes_exp

                if not new_plan:
                    print(f"[Time {simulation_time}] Replanning failed. No path found.")
                    path = None # Indicate failure
                    break

                current_plan = new_plan
                print(f"New plan: {current_plan}")
                # Take the first step of the new plan
                next_step = current_plan.pop(0)

            # Move to the next step
            current_pos = next_step
            path.append(current_pos)
            environment.update_agent_position(current_pos) # For visualization/logging
            print(f"[Time {simulation_time}] Agent moved to {current_pos}")

        expanded_nodes = total_expanded_nodes # For reporting


    end_time = time.time()

    # --- Report Results ---
    print("\n--- RESULTS ---")
    if path:
        path_cost = agent.calculate_path_cost(path)
        print(f"Path Found: {' -> '.join(map(str, path))}")
        print(f"Path Cost: {path_cost}")
        print(f"Path Length: {len(path) - 1} steps")
    else:
        print("Path not found!")

    print(f"Nodes Expanded: {expanded_nodes}")
    print(f"Execution Time: {end_time - start_time:.6f} seconds")
    print("---------------")


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'environment'